In [29]:
import os

from datasets import flowers
from nets import resnet_v1
from nets import resnet_utils
from preprocessing import vgg_preprocessing
from keras import utils

from keras.applications.vgg16 import (
    VGG16, preprocess_input, decode_predictions)
from keras.preprocessing import image
import tensorflow as tf
from tensorflow.contrib import slim
import numpy as np

In [7]:
class CustomDataGen():

    def __init__(self, dim_x, dim_y, dim_z, num_class, batch_size):
        self.batch_size = batch_size
        self.dim_x = dim_x
        self.dim_y = dim_y
        self.dim_z = dim_z
        self.num_class = num_class
        # self.augmentation = image.ImageDataGenerator(
        #     rotation_range=20,
        #     shear_range=0.5
        # )

    def randomize_ind(self,data):
        indexes = np.arange(len(data))
        np.random.shuffle(indexes)
        return indexes

    def get_data(self,list):

        X = np.empty((self.batch_size, self.dim_x, self.dim_y, self.dim_z))
        y = np.empty((self.batch_size,self.num_class))

        for id, data in enumerate(list):
            im_path = data.split(' ')[0]
            label = int(data.split(' ')[1])
            img = image.load_img(im_path, target_size=(self.dim_x, self.dim_y))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)[0]
            X[id,:,:,:] = x

            y_ = utils.to_categorical(label, self.num_class)
            y[id,...] = y_

        return X, y

    def generate_batch(self, data):

        while 1:
            indexes = self.randomize_ind(data)

            num_batch = int(len(indexes)/self.batch_size)
            for batch_id in range(num_batch):
                temp_list = [data[k] for k in indexes[batch_id*self.batch_size:(batch_id+1)*self.batch_size]]

                X,y = self.get_data(temp_list)
                # return self.augmentation.flow(X,y,self.batch_size)
                yield X,y


In [50]:
def read_img_list_from_file(file_path):

    data = []
    with open(file_path) as f:
        for line in f:
            data.append(line)

    return data

num_classes = 27
batch_size = 30
train_data = read_img_list_from_file('/Users/vidit/Thesis/Grocery_products/train_resnet/train.txt')


In [79]:
training_generator = CustomDataGen(224, 224, 3, num_classes, batch_size).generate_batch(train_data)
image_size = resnet_v1.resnet_v1_101.default_image_size

train_dir = '/Users/vidit/Thesis/Grocery_products/train_resnet/'
def get_init_fn():
    """Returns a function run by the chief worker to warm-start the training."""
    checkpoint_exclude_scopes=["resnet_v1_101/logits"]
    
    exclusions = [scope.strip() for scope in checkpoint_exclude_scopes]

    variables_to_restore = []
    for var in slim.get_model_variables():
        excluded = False
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
                break
        if not excluded:
            variables_to_restore.append(var)

    return slim.assign_from_checkpoint_fn(
      os.path.join(train_dir, 'resnet_v1_101.ckpt'),
      variables_to_restore)



with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
    
    
    images = tf.placeholder(tf.float32, shape=(batch_size, 224, 224, 3))
    labels = tf.placeholder(tf.float32, shape=(batch_size, 27))
    
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(resnet_v1.resnet_arg_scope()):
        logits, _ = resnet_v1.resnet_v1_101(images, num_classes=num_classes, is_training=True)

    # Specify the loss function:
    slim.losses.softmax_cross_entropy(logits, labels)
    total_loss = slim.losses.get_total_loss()

    # Create some summaries to visualize the training process:
    tf.summary.scalar('losses/Total_Loss', total_loss)

    # Specify the optimizer and create the train op:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = slim.learning.create_train_op(total_loss, optimizer)
    init_fn = get_init_fn();

    with tf.Session() as sess:
        
        sess.run(tf.initialize_all_variables())
        init_fn(sess)
        for b_images,b_labels in training_generator:
            print('run')
            feed_dict = {images:b_images,labels:b_labels}
            final_loss,_ = sess.run([total_loss,train_op],feed_dict=feed_dict)



            print('Finished training. Last batch loss %f' % final_loss)
            break

Instructions for updating:
Use tf.losses.softmax_cross_entropy instead. Note that the order of the logits and labels arguments has been changed.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
Instructions for updating:
Use tf.losses.get_total_loss instead.
Instructions for updating:
Use tf.losses.get_losses instead.
Instructions for updating:
Use tf.losses.get_regularization_losses instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
INFO:tensorflow:Restoring parameters from /Users/vidit/Thesis/Grocery_products/train_resnet/resnet_v1_101.ckpt
run
Finished training. Last batch loss 3.825681
